In [ ]:
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
!pip3 install sentence-transformers
from sentence_transformers import SentenceTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 3.2 MB/s 
     |████████████████████████████████| 4.2 MB 9.1 MB/s 
     |████████████████████████████████| 1.2 MB 53.3 MB/s 
     |████████████████████████████████| 86 kB 4.9 MB/s 
     |████████████████████████████████| 596 kB 59.9 MB/s 
     |████████████████████████████████| 6.6 MB 58.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=3564e5536013318d82cf6deda2373246958ea1232b2418bb04254b9125242cb8
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from zipfile import ZipFile
file_name="/content/drive/MyDrive/book.zip"
with ZipFile (file_name,'r') as zip:
   zip.extractall()
   print('Done')

Done


In [ ]:
#Load pre-trained SentenceBERT model
bert = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
book = pd.read_csv('Books.csv', header=0)

In [ ]:
books =  book.sample(frac=0.02)

In [ ]:
books.head(1)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
75080,0394499603,In the Beginning,Chaim Potok,1975,Random House Inc,http://images.amazon.com/images/P/0394499603.0...,http://images.amazon.com/images/P/0394499603.0...,http://images.amazon.com/images/P/0394499603.0...


In [ ]:
books['Book-Author'].fillna(books['Book-Title'],inplace=True)

In [ ]:
books['Publisher'].fillna(books['Book-Title'],inplace=True)

In [ ]:
rating = pd.read_csv('Ratings.csv', header=0)
user = pd.read_csv('Users.csv', header=0)

In [ ]:
rating1 = rating[rating.ISBN.isin(books.ISBN)]
#ratings_new.head()

In [ ]:
rating1 = rating1[rating1['Book-Rating']!=0]

In [ ]:
for x in books['ISBN']:
    print(type(x))
    break

<class 'str'>


In [ ]:
rating1['Book-Rating']==0

145        False
215        False
329        False
418        False
628        False
           ...  
1149502    False
1149570    False
1149586    False
1149588    False
1149738    False
Name: Book-Rating, Length: 7808, dtype: bool

In [ ]:
rating1.head(10)

,User-ID,ISBN,Book-Rating
145,276822,0786817070,10
215,276847,3608932240,7
329,276872,0425188221,7
418,276925,0140154078,6
628,276939,2070419657,7
735,276986,0375760911,10
781,277018,0451097009,7
890,277134,1586609726,6
897,277143,031286504X,10
917,277157,0345452550,7


In [ ]:
rating1.shape

(7808, 3)

In [ ]:
books['Year-Of-Publication'] =books['Year-Of-Publication'].astype(int)

In [ ]:
threshold_date = 2022
books = books[books['Year-Of-Publication'] < threshold_date]

In [ ]:
books.head(1)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
75080,0394499603,In the Beginning,Chaim Potok,1975,Random House Inc,http://images.amazon.com/images/P/0394499603.0...,http://images.amazon.com/images/P/0394499603.0...,http://images.amazon.com/images/P/0394499603.0...


In [ ]:
#1149780
threshold_date = 1998
ids = books[books['Year-Of-Publication'] < threshold_date]['ISBN'].values

training_data = rating1[rating1['ISBN'].isin(ids)]
print(f'Training data size: {training_data.shape}')

#~ is not in ratings
testing_data = rating1[~rating1['ISBN'].isin(ids)]
print(f'Testing data size: {testing_data.shape}')

Training data size: (3770, 3)
Testing data size: (4038, 3)


In [ ]:
testing_data = testing_data[testing_data['User-ID'].isin(training_data['User-ID'])]
testing_data.shape

(1188, 3)

In [ ]:
training_data.head()

,User-ID,ISBN,Book-Rating
418,276925,0140154078,6
781,277018,0451097009,7
897,277143,031286504X,10
956,277157,0684801663,6
1202,277203,030700645X,8


#Model Evaluator model 

In [ ]:
def get_favorite_books(user_id, ratings_df):
    favorites = ratings_df[(ratings_df['User-ID'] == user_id) & (ratings_df['Book-Rating'] >= 6.5)].sort_values(by='Book-Rating', ascending=False)['ISBN']
    return set(favorites if type(favorites) == pd.Series else [favorites])

In [207]:
class ModelEvaluator:
  def __init__(self, training_data, testing_data, threshold=5.5):
    self.training_data = training_data
    self.testing_data = testing_data
    self.threshold = threshold


  def evaluate(self, model, user_id):
      #Getting the items in test set
      #print(user_id)
      favorites_in_test = get_favorite_books(user_id, self.testing_data)
      #print(favorites_in_test)
      
      #Getting a ranked recommendation list from a model for a given user (movieId, predicted_rating)
      person_recs_df = model.recommend_items(user_id, 
                                              items_to_ignore=get_favorite_books(user_id, self.training_data))
      #print(person_recs_df['predicted_rating'] )

      if not person_recs_df.empty:
        person_recs_df = person_recs_df[person_recs_df['predicted_rating'] >= self.threshold].sort_values(by='predicted_rating', ascending=False)

      return person_recs_df


  def evaluate_model_for_user(self, model, user_id):
      #Getting the items in test set
      favorites_in_test = get_favorite_books(user_id, self.testing_data)
      #print(favorites_in_test)
      
      #Getting a ranked recommendation list from a model for a given user (movieId, predicted_rating)
      person_recs_df = model.recommend_items(user_id, 
                                              items_to_ignore=get_favorite_books(user_id, self.training_data))
     # print(person_recs_df)
      # get only movies with predicated rating >= 3.5
      person_recs_df = person_recs_df[person_recs_df['predicted_rating'] >= self.threshold].sort_values(by='predicted_rating', ascending=False)
      true_relevent = person_recs_df[person_recs_df['ISBN'].isin(favorites_in_test)].shape[0]

      top_5_recommended = person_recs_df.head(5)
      top_10_recommended = person_recs_df.head(10)

      # Number of relevant and recommended items in top k
      hits_at_5_count = top_5_recommended[top_5_recommended['ISBN'].isin(favorites_in_test)].shape[0]
      hits_at_10_count = top_10_recommended[top_10_recommended['ISBN'].isin(favorites_in_test)].shape[0]

      precision_at_5 = hits_at_5_count / top_5_recommended.shape[0] if top_5_recommended.shape[0] != 0 else 1
      recall_at_5 = hits_at_5_count / true_relevent if true_relevent != 0 else 1

      precision_at_10 = hits_at_10_count / top_10_recommended.shape[0] if top_10_recommended.shape[0] != 0 else 1
      recall_at_10 = hits_at_10_count / true_relevent if true_relevent != 0 else 1

      person_metrics = {'hits@5_count':hits_at_5_count, 
                        'hits@10_count':hits_at_10_count,
                        'recommended@5_count':top_5_recommended.shape[0], 
                        'recommended@10_count':top_10_recommended.shape[0],  
                        'relevents': true_relevent,
                        'recall@5': recall_at_5,
                        'recall@10': recall_at_10,
                        'precision@5': precision_at_5,
                        'precision@10': precision_at_10}
      return person_metrics


  def evaluate_model(self, model):
      #print('Running evaluation for users')
      users_metrics = []
      users_ids = list(set(self.testing_data['User-ID'].values))
      for idx, user_id in enumerate(users_ids):
          metrics = self.evaluate_model_for_user(model, user_id)  
          metrics['User-ID'] = user_id
          users_metrics.append(metrics)
      print('%d users processed' % idx)

      detailed_results_df = pd.DataFrame(users_metrics) \
                          .sort_values('hits@5_count', ascending=False)
      
      global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['relevents'].sum())
      global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['relevents'].sum())
      
      global_precision_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['recommended@5_count'].sum())
      global_precision_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['recommended@10_count'].sum())

      global_metrics = {'modelName': model.get_model_name(),
                        'recall@5': global_recall_at_5,
                        'recall@10': global_recall_at_10,
                        'precision@5': global_precision_at_5,
                        'precision@10': global_precision_at_10}    
      return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator(training_data, testing_data)              

##Content Based

In [ ]:
print(len(books['Book-Author'].unique()))

4659


In [ ]:
def check_name(x):
  words = x.split()
  return all(x[0].isupper() and x[1].islower() for x in words if len(x) > 1)

def clean(x):
  if x.istitle() or check_name(x):
    return x.replace(" ", "").lower()
  else:
    return x.lower().strip()

In [ ]:
books['Book-Author'] = books['Book-Author'].apply(lambda x: clean(x))

In [ ]:
#books['Book-Title'] = books['Book-Title'].apply(lambda x: clean(x))

In [ ]:
books['Publisher'] = books['Publisher'].apply(lambda x: clean(x))

In [ ]:
# get all unique genres
publishers = list(set([x for publishers in books['Book-Author'].values for x in publishers.split('|')]))
#publishers

In [ ]:
pub = list(set([x for publishers in books['Book-Title'].values for x in publishers.split('|')]))

In [ ]:
pub1 = list(set([x for publishers in books['Publisher'].values for x in publishers.split('|')]))

In [ ]:
expanded_books_df = books.copy()
for p in publishers:
  expanded_books_df[p] = [0 if not p in publishers.split('|') else 1 for publishers in books['Book-Author'].values]

In [ ]:
expanded_books_df1 = books.copy()
for p in pub:
  expanded_books_df1[p] = [0 if not p in publishers.split('|') else 1 for publishers in books['Book-Title'].values]

In [ ]:
expanded_books_df2 = books.copy()
for p in pub1:
  expanded_books_df2[p] = [0 if not p in publishers.split('|') else 1 for publishers in books['Publisher'].values]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def normalize(value, old_max, old_min, new_max=10.0, new_min=0.0):
  old_range = (old_max - old_min)  
  new_range = (new_max - new_min)  
  return (((value - old_min) * new_range) / old_range) + new_min

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(expanded_books_df['Book-Author'])
count_matrix.shape

(5427, 5215)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidfVector = tfidf.fit_transform(expanded_books_df['Book-Author'])
tfidfVector.shape

(5427, 5215)

In [ ]:
tfidfVector1 = tfidf.fit_transform(expanded_books_df['Book-Title'])
tfidfVector1.shape

(5427, 8669)

In [ ]:
tfidfVector2 = tfidf.fit_transform(expanded_books_df['Publisher'])
tfidfVector2.shape

(5427, 1938)

In [ ]:
#count = CountVectorizer(stop_words='english')
count_matrix1 = count.fit_transform(expanded_books_df1['Book-Title'])
count_matrix1.shape

(5427, 8669)

In [ ]:
count_matrix2 = count.fit_transform(expanded_books_df2['Publisher'])
count_matrix2.shape

(5427, 1938)

In [ ]:
#sentence_embeddings1 = bert.encode(books['Book-Author'].tolist())

In [ ]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, items_ids, items_matrix, training_data, testing_data):
        self.items_ids = items_ids
        self.items_matrix = items_matrix
        self.training_data = training_data
        self.testing_data = testing_data
        
    def get_model_name(self):
        return self.MODEL_NAME

    def get_item_profile(self, item_id):
        #print(item_id)
        idx = self.items_ids.index(item_id)
        
        return self.items_matrix[idx].toarray().reshape(-1)

    def get_items_profiles(self, ids):
        items_profiles = np.array([self.get_item_profile(x) for x in ids])
        return items_profiles

    def build_users_profile(self, user_id):
        user_df = self.training_data[self.training_data['User-ID'] == user_id]
        user_items_profiles = self.get_items_profiles(user_df['ISBN'].values)
        #print(user_id)
        #print(user_df)
        #print(user_items_profiles)
        user_items_ratings = np.array(user_df['Book-Rating'].values).reshape(-1,1)
        user_profile = np.sum(np.multiply(user_items_profiles, user_items_ratings), axis=0) / np.sum(user_items_ratings)
        #print(user_profile)
        return user_profile

    def get_similar_items_to_user_profile(self, user_id, topn=1000):
        user_profile = self.build_users_profile(user_id).reshape(1, -1)
        #Computes the cosine similarity between the user profile and all item profiles
        cosine_similarities = cosine_similarity(user_profile, self.items_matrix.toarray())
        #Gets the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        #Sort the similar items by similarity
        similar_items = sorted([(self.items_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10):
        similar_items = self.get_similar_items_to_user_profile(user_id)
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['ISBN', 'predicted_rating']) \
                                    .head(topn)

        
        #print(user_id)
        recommendations_df['predicted_rating'] = recommendations_df['predicted_rating'].apply(lambda x: normalize(x, 1.0, 0.0))
        return recommendations_df
    
content_based_recommender_model = ContentBasedRecommender(expanded_books_df['ISBN'].values.tolist(), count_matrix, training_data, testing_data)

In [ ]:
content_based_recommender_model1 = ContentBasedRecommender(expanded_books_df1['ISBN'].values.tolist(), count_matrix1, training_data, testing_data)

In [ ]:
content_based_recommender_model2 = ContentBasedRecommender(expanded_books_df2['ISBN'].values.tolist(), count_matrix2, training_data, testing_data)

In [ ]:
content_based_recommender_model3 = ContentBasedRecommender(expanded_books_df['ISBN'].values.tolist(), tfidfVector, training_data, testing_data)

In [ ]:
content_based_recommender_model4 = ContentBasedRecommender(expanded_books_df['ISBN'].values.tolist(), tfidfVector1, training_data, testing_data)

In [ ]:
content_based_recommender_model5 = ContentBasedRecommender(expanded_books_df['ISBN'].values.tolist(), tfidfVector2, training_data, testing_data)

##Book Author

In [ ]:
#276925
metrics = model_evaluator.evaluate(content_based_recommender_model3, 277018)
#cb_global_metrics, cb_detailed_results_df,based = model_evaluator.evaluate_model(content_based_recommender_model3)

set()
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
7    0.0
8    0.0
9    0.0
Name: predicted_rating, dtype: float64


In [ ]:
metrics

,ISBN,predicted_rating


In [ ]:
#fav =model_evaluator.evaluate(content_based_recommender_model3)

In [ ]:
print('Evaluating Content-Based Filtering model...')
cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model3)
print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)

Evaluating Content-Based Filtering model...
573 users processed

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.9, 'recall@10': 1.0, 'precision@5': 0.013138686131386862, 'precision@10': 0.0125}


,hits@5_count,hits@10_count,recommended@5_count,recommended@10_count,relevents,recall@5,recall@10,precision@5,precision@10,User-ID
464,2,2,2,2,2,1.0,1.0,1.000000,1.000000,274061
108,1,1,5,8,1,1.0,1.0,0.200000,0.125000,110973
86,1,1,2,2,1,1.0,1.0,0.500000,0.500000,168245
49,1,1,5,8,1,1.0,1.0,0.200000,0.125000,104636
174,1,1,2,2,1,1.0,1.0,0.500000,0.500000,172676
493,1,1,3,3,1,1.0,1.0,0.333333,0.333333,67322
209,1,1,5,10,1,1.0,1.0,0.200000,0.100000,236283
193,1,1,2,2,1,1.0,1.0,0.500000,0.500000,8890
383,0,0,1,1,0,1.0,1.0,0.000000,0.000000,38281
384,0,0,5,8,0,1.0,1.0,0.000000,0.000000,189835


In [ ]:
print('Evaluating Content-Based Filtering model...')
cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model)
print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)

Evaluating Content-Based Filtering model...
573 users processed

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.9, 'recall@10': 1.0, 'precision@5': 0.01347305389221557, 'precision@10': 0.01278772378516624}


,hits@5_count,hits@10_count,recommended@5_count,recommended@10_count,relevents,recall@5,recall@10,precision@5,precision@10,User-ID
464,2,2,2,2,2,1.0,1.0,1.000000,1.000000,274061
108,1,1,5,8,1,1.0,1.0,0.200000,0.125000,110973
86,1,1,2,2,1,1.0,1.0,0.500000,0.500000,168245
49,1,1,5,8,1,1.0,1.0,0.200000,0.125000,104636
174,1,1,2,2,1,1.0,1.0,0.500000,0.500000,172676
493,1,1,3,3,1,1.0,1.0,0.333333,0.333333,67322
209,1,1,5,10,1,1.0,1.0,0.200000,0.100000,236283
193,1,1,2,2,1,1.0,1.0,0.500000,0.500000,8890
383,0,0,1,1,0,1.0,1.0,0.000000,0.000000,38281
384,0,0,5,8,0,1.0,1.0,0.000000,0.000000,189835


##Book Title

In [ ]:
print('Evaluating Content-Based Filtering model...')
cb_global_metrics1, cb_detailed_results_df1 = model_evaluator.evaluate_model(content_based_recommender_model1)
print('\nGlobal metrics:\n%s' % cb_global_metrics1)
cb_detailed_results_df1.head(10)

Evaluating Content-Based Filtering model...
573 users processed

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 1.0, 'recall@10': 1.0, 'precision@5': 0.003067484662576687, 'precision@10': 0.003003003003003003}


,hits@5_count,hits@10_count,recommended@5_count,recommended@10_count,relevents,recall@5,recall@10,precision@5,precision@10,User-ID
493,1,1,2,2,1,1.0,1.0,0.5,0.5,67322
377,0,0,4,4,0,1.0,1.0,0.0,0.0,267635
379,0,0,1,1,0,1.0,1.0,0.0,0.0,25981
380,0,0,0,0,0,1.0,1.0,1.0,1.0,38273
381,0,0,0,0,0,1.0,1.0,1.0,1.0,112001
382,0,0,3,3,0,1.0,1.0,0.0,0.0,243075
383,0,0,0,0,0,1.0,1.0,1.0,1.0,38281
384,0,0,0,0,0,1.0,1.0,1.0,1.0,189835
385,0,0,0,0,0,1.0,1.0,1.0,1.0,132492
0,0,0,2,2,0,1.0,1.0,0.0,0.0,249862


In [ ]:
print('Evaluating Content-Based Filtering model...')
cb_global_metrics4, cb_detailed_results_df4 = model_evaluator.evaluate_model(content_based_recommender_model4)
print('\nGlobal metrics:\n%s' % cb_global_metrics4)
cb_detailed_results_df4.head(10)

Evaluating Content-Based Filtering model...
573 users processed

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.6666666666666666, 'recall@10': 1.0, 'precision@5': 0.0010610079575596816, 'precision@10': 0.0010905125408942203}


,hits@5_count,hits@10_count,recommended@5_count,recommended@10_count,relevents,recall@5,recall@10,precision@5,precision@10,User-ID
493,1,1,2,2,1,1.0,1.0,0.5,0.500000,67322
49,1,1,5,6,1,1.0,1.0,0.2,0.166667,104636
378,0,0,5,10,0,1.0,1.0,0.0,0.000000,198012
380,0,0,4,4,0,1.0,1.0,0.0,0.000000,38273
381,0,0,5,10,0,1.0,1.0,0.0,0.000000,112001
382,0,0,5,8,0,1.0,1.0,0.0,0.000000,243075
383,0,0,3,3,0,1.0,1.0,0.0,0.000000,38281
384,0,0,3,3,0,1.0,1.0,0.0,0.000000,189835
385,0,0,0,0,0,1.0,1.0,1.0,1.000000,132492
0,0,0,5,10,0,1.0,1.0,0.0,0.000000,249862


##Publisher

In [ ]:
print('Evaluating Content-Based Filtering model...')
cb_global_metrics2, cb_detailed_results_df2 = model_evaluator.evaluate_model(content_based_recommender_model2)
print('\nGlobal metrics:\n%s' % cb_global_metrics2)
cb_detailed_results_df2.head(10)

Evaluating Content-Based Filtering model...
573 users processed

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.25, 'recall@10': 1.0, 'precision@5': 0.00043535045711797995, 'precision@10': 0.0009269988412514484}


,hits@5_count,hits@10_count,recommended@5_count,recommended@10_count,relevents,recall@5,recall@10,precision@5,precision@10,User-ID
86,1,1,5,10,1,1.0,1.0,0.2,0.1,168245
0,0,0,1,1,0,1.0,1.0,0.0,0.0,249862
378,0,0,5,10,0,1.0,1.0,0.0,0.0,198012
380,0,0,5,10,0,1.0,1.0,0.0,0.0,38273
381,0,0,5,10,0,1.0,1.0,0.0,0.0,112001
382,0,0,5,10,0,1.0,1.0,0.0,0.0,243075
383,0,0,5,10,0,1.0,1.0,0.0,0.0,38281
384,0,0,5,10,0,1.0,1.0,0.0,0.0,189835
385,0,0,0,0,0,1.0,1.0,1.0,1.0,132492
386,0,0,1,1,0,1.0,1.0,0.0,0.0,206219


In [ ]:
print('Evaluating Content-Based Filtering model...')
cb_global_metrics5, cb_detailed_results_df5 = model_evaluator.evaluate_model(content_based_recommender_model5)
print('\nGlobal metrics:\n%s' % cb_global_metrics5)
cb_detailed_results_df5.head(10)

Evaluating Content-Based Filtering model...
573 users processed

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.3333333333333333, 'recall@10': 1.0, 'precision@5': 0.0007993605115907274, 'precision@10': 0.0012581253931641854}


,hits@5_count,hits@10_count,recommended@5_count,recommended@10_count,relevents,recall@5,recall@10,precision@5,precision@10,User-ID
86,1,1,5,10,1,1.0,1.0,0.2,0.1,168245
510,1,1,5,10,1,1.0,1.0,0.2,0.1,128835
0,0,0,1,1,0,1.0,1.0,0.0,0.0,249862
385,0,0,0,0,0,1.0,1.0,1.0,1.0,132492
379,0,0,5,10,0,1.0,1.0,0.0,0.0,25981
380,0,0,5,10,0,1.0,1.0,0.0,0.0,38273
381,0,0,5,10,0,1.0,1.0,0.0,0.0,112001
382,0,0,5,10,0,1.0,1.0,0.0,0.0,243075
383,0,0,5,10,0,1.0,1.0,0.0,0.0,38281
384,0,0,5,10,0,1.0,1.0,0.0,0.0,189835


#Collaborative Filtering model

In [ ]:
users_items_pivot_matrix_df = rating1.pivot(index='User-ID', columns='ISBN', values='Book-Rating').fillna(0)
users_items_pivot_matrix_df.shape

(5131, 2991)

In [ ]:
users_items_pivot_matrix = users_items_pivot_matrix_df.values
users_items_pivot_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
from scipy.sparse.linalg import svds

#The number of factors to factor the user-item matrix.
NUMBER_OF_FACTORS_MF = 15
#Performs matrix factorization of the original user item matrix
#U, sigma, Vt = svds(users_items_pivot_matrix, k = NUMBER_OF_FACTORS_MF)
U, sigma, Vt = svds(users_items_pivot_matrix, k = NUMBER_OF_FACTORS_MF)

In [ ]:
sigma = np.diag(sigma)
sigma.shape

(15, 15)

In [ ]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_ratings

array([[ 1.76184957e-20, -3.77440840e-21,  1.09840394e-04, ...,
        -9.97498110e-05,  1.35041158e-19,  4.29204932e-21],
       [ 8.10242092e-19, -1.12687503e-19,  6.87169520e-04, ...,
         2.38397279e-04, -3.16744220e-18, -1.43966589e-19],
       [-3.25945457e-19,  2.14170083e-20, -4.25962482e-05, ...,
         8.58420600e-06,  5.04545367e-19,  5.60604769e-20],
       ...,
       [-1.05142025e-17,  3.87609811e-19,  4.75086299e-03, ...,
        -1.88001917e-03, -3.22540339e-17,  3.27374486e-18],
       [ 3.97242509e-19, -6.93668660e-20,  4.78830081e-04, ...,
         2.69429899e-04, -2.61725831e-18, -3.90934744e-20],
       [-1.05142025e-17,  3.87609811e-19,  4.75086299e-03, ...,
        -1.88001917e-03, -3.22540339e-17,  3.27374486e-18]])

In [ ]:
#Converting the reconstructed matrix back to a Pandas dataframe
preds_df = pd.DataFrame(all_user_predicted_ratings, columns=users_items_pivot_matrix_df.columns, index=users_items_pivot_matrix_df.index)

In [ ]:
preds_df.head(10)

ISBN,0004105028,0004333349,0004704746,0006479499,0006922007,0007101953,0007107900,0007116497,0007143907,0020102011,...,9679966054,9684322046,9722109332,9724115380,9725681398,9757054119,9813056517,9997405137,9999980538,B00005UMI4
User-ID,,,,,,,,,,,,,,,,,,,,,
69,1.761850e-20,-3.774408e-21,1.098404e-04,-4.835728e-19,-1.209558e-06,-7.239323e-06,-1.179694e-04,-1.183745e-04,-2.434521e-19,1.614446e-19,...,-2.906897e-20,-1.150327e-19,-1.909040e-19,5.017379e-19,5.749616e-19,-2.242544e-20,-1.251841e-19,-9.974981e-05,1.350412e-19,4.292049e-21
226,8.102421e-19,-1.126875e-19,6.871695e-04,3.290120e-18,2.453878e-05,1.133869e-04,6.246165e-02,-6.219419e-04,8.176885e-19,-8.865797e-19,...,-1.057761e-18,-1.009776e-18,2.027087e-18,-4.791848e-18,-5.900344e-18,2.619090e-19,3.980229e-19,2.383973e-04,-3.167442e-18,-1.439666e-19
257,-3.259455e-19,2.141701e-20,-4.259625e-05,8.737130e-19,6.191108e-07,-2.459869e-06,-6.318983e-04,7.317257e-06,1.559269e-19,-1.226467e-19,...,4.533654e-19,4.619389e-19,-2.237708e-19,-3.330225e-19,-4.839206e-19,-1.375259e-19,-3.712152e-19,8.584206e-06,5.045454e-19,5.606048e-20
310,-1.019370e-33,-4.033550e-35,-7.109281e-20,9.164603e-34,-1.319207e-20,-3.758832e-20,7.956197e-19,-8.474352e-20,-3.053909e-34,1.034330e-34,...,2.422515e-33,9.666689e-35,-1.739250e-33,3.416900e-35,9.569933e-35,-4.777374e-34,-1.380832e-33,-1.219540e-19,6.529653e-35,2.925696e-34
383,5.524981e-20,-1.248765e-19,6.312103e-04,2.422425e-19,-3.440880e-05,7.200020e-06,5.518041e-02,-4.858898e-04,-1.525414e-19,-3.220074e-19,...,4.050123e-19,-1.172598e-18,2.327966e-19,1.604578e-18,8.710520e-19,-8.777880e-20,-3.079821e-19,2.586845e-04,-2.959997e-18,6.338095e-20
387,-6.112101e-20,-7.139200e-21,1.236484e-05,1.671896e-19,-3.642323e-06,-1.036161e-05,9.010039e-04,-5.941684e-06,-4.704725e-20,4.335385e-20,...,1.529619e-19,-3.467860e-20,-7.921684e-20,-1.592566e-19,-1.904982e-19,-3.045176e-20,-7.922328e-20,-8.941641e-06,4.937730e-20,1.955616e-20
444,-3.712810e-19,-3.970833e-20,-8.940263e-05,1.995734e-18,4.090383e-04,9.234661e-04,-2.976089e-03,4.009599e-05,-6.319790e-20,-2.019129e-19,...,6.901463e-19,1.966384e-19,-4.341927e-19,-2.295078e-18,-3.101088e-19,-2.369759e-19,1.489432e-19,6.848423e-05,-3.309130e-19,2.503443e-20
486,4.405900e-17,1.383549e-18,-1.141750e-03,-1.647525e-17,9.180011e-06,-1.031840e-03,-9.884890e-03,6.929950e-04,-1.931516e-17,2.611893e-17,...,-1.020267e-16,-3.568623e-18,8.408081e-17,-3.018259e-18,-1.621666e-18,1.960315e-17,5.995914e-17,1.537021e-03,4.242859e-18,-1.265199e-17
505,-2.004413e-18,4.006244e-20,1.218285e-04,3.877436e-18,4.372456e-06,1.988463e-04,-5.015071e-03,3.397053e-04,6.271919e-19,-8.212648e-19,...,4.864236e-18,1.870314e-18,-3.063715e-18,-3.108685e-18,-2.913435e-18,-8.537218e-19,-3.269889e-18,8.449588e-05,-2.391249e-19,6.223295e-19


In [ ]:
preds_df = preds_df.apply(lambda x: normalize(x, all_user_predicted_ratings.max(), all_user_predicted_ratings.min()))
#preds_df.head(10)

In [ ]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, predictions_df):
        self.predictions_df = predictions_df
        
    def get_model_name(self):
        return self.MODEL_NAME

    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Get and sort the user's predictions
        sorted_user_predictions = self.predictions_df.loc[user_id].sort_values(ascending=False)
        recommendations = {'ISBN': sorted_user_predictions.index, 'predicted_rating': sorted_user_predictions.values}
        recommendations_df = pd.DataFrame(recommendations)
        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        recommendations_df = recommendations_df[~recommendations_df['ISBN'].isin(items_to_ignore)] \
                               .sort_values('predicted_rating', ascending = False) \
                               .head(topn)

        return recommendations_df
    
cf_recommender_model = CFRecommender(preds_df)

In [ ]:
model_evaluator1 = ModelEvaluator(training_data, testing_data, 2.5) 

In [ ]:
cf_global_metrics1, cf_detailed_results_df1,bosed1 = model_evaluator.evaluate_model(cf_recommender_model)

573 users processed


In [ ]:
bosed1[0]

,ISBN,predicted_rating
0,051513628X,10.0


In [ ]:
model_evaluator = ModelEvaluator(training_data, testing_data, 2.5)              

print('Evaluating Collaborative Filtering (SVD Matrix Factorization) model...')
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(cf_recommender_model)
print('\nGlobal metrics:\n%s' % cf_global_metrics)
cf_detailed_results_df.head(10)

Evaluating Collaborative Filtering (SVD Matrix Factorization) model...
573 users processed

Global metrics:
{'modelName': 'Collaborative Filtering', 'recall@5': 0.8952380952380953, 'recall@10': 1.0, 'precision@5': 0.5911949685534591, 'precision@10': 0.6069364161849711}


,hits@5_count,hits@10_count,recommended@5_count,recommended@10_count,relevents,recall@5,recall@10,precision@5,precision@10,User-ID
13,5,10,5,10,10,0.500000,1.0,1.0,1.000000,153662
197,5,6,5,9,6,0.833333,1.0,1.0,0.666667,76499
388,5,10,5,10,10,0.500000,1.0,1.0,1.000000,11676
372,3,3,3,3,3,1.000000,1.0,1.0,1.000000,25966
105,2,2,2,2,2,1.000000,1.0,1.0,1.000000,76151
185,2,2,2,2,2,1.000000,1.0,1.0,1.000000,37538
141,2,2,2,2,2,1.000000,1.0,1.0,1.000000,170513
200,2,2,2,2,2,1.000000,1.0,1.0,1.000000,78553
116,2,2,4,4,2,1.000000,1.0,0.5,0.500000,16795
91,2,2,2,2,2,1.000000,1.0,1.0,1.000000,241980


#hybrid  model

In [283]:
class hybridRecommender:
    
    MODEL_NAME = 'hybrid'
    
    #def __init__(self):
     #   self.predictions_df = predictions_df
        
    def get_model_name(self):
        return self.MODEL_NAME

    def recommend_items(self, user_id):
      #metrics = self.evaluate_model_for_user(model, user_id)

        metrics1 = model_evaluator.evaluate(content_based_recommender_model3, user_id)
        
        #print(metrics1['ISBN'])

        metrics2 =model_evaluator1.evaluate(cf_recommender_model,user_id)


        frames = [metrics1, metrics2]
  
        metrics = pd.concat(frames)


        s1 = pd.merge(metrics1, metrics2, how='inner', on=['ISBN'])

        #if s1.empty:
        metrics = metrics.sort_values('predicted_rating',ascending=False)
          #print(metrics)

        #else:
           #print(s1)
           # if row['predicted_rating']>3.5:
          
         # metrics = s1
        return metrics
    
hybridRecommender_model = hybridRecommender()

In [284]:
users_metrics = []
i = 0 
users_ids = list(set(testing_data['User-ID'].values))
for idx, user_id in enumerate(users_ids):
             favorites_in_test = get_favorite_books(user_id,testing_data)
             metrics= hybridRecommender_model.recommend_items(user_id)
             #print(metrics)
             true_relevent = metrics[metrics['ISBN'].isin(favorites_in_test)].shape[0]

             top_5_recommended = metrics.head(5)
             top_10_recommended = metrics.head(10)
             hits_at_5_count = top_5_recommended[top_5_recommended['ISBN'].isin(favorites_in_test)].shape[0]
             hits_at_10_count = top_10_recommended[top_10_recommended['ISBN'].isin(favorites_in_test)].shape[0]

             precision_at_5 = hits_at_5_count / top_5_recommended.shape[0] if top_5_recommended.shape[0] != 0 else 1
             recall_at_5 = hits_at_5_count / true_relevent if true_relevent != 0 else 1

             precision_at_10 = hits_at_10_count / top_10_recommended.shape[0] if top_10_recommended.shape[0] != 0 else 1
             recall_at_10 = hits_at_10_count / true_relevent if true_relevent != 0 else 1

             person_metrics = {'hits@5_count':hits_at_5_count, 
                        'hits@10_count':hits_at_10_count,
                        'recommended@5_count':top_5_recommended.shape[0], 
                        'recommended@10_count':top_10_recommended.shape[0],  
                        'relevents': true_relevent,
                        'recall@5': recall_at_5,
                        'recall@10': recall_at_10,
                        'precision@5': precision_at_5,
                        'precision@10': precision_at_10}
             person_metrics['User-ID'] = user_id
             users_metrics.append(person_metrics)

print('%d users processed' % idx)
detailed_results_df = pd.DataFrame(users_metrics) \
                          .sort_values('hits@5_count', ascending=False)
      
global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['relevents'].sum())
global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['relevents'].sum())
      
global_precision_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['recommended@5_count'].sum())
global_precision_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['recommended@10_count'].sum())

global_metrics = {'modelName': hybridRecommender_model.get_model_name(),
                        'recall@5': global_recall_at_5,
                        'recall@10': global_recall_at_10,
                        'precision@5': global_precision_at_5,
                        'precision@10': global_precision_at_10}

Streaming output truncated to the last 5000 lines.
         ISBN  predicted_rating
0  0375703063          0.999716
1  0446532231          0.997419
2  0380002930          0.997186
3  0060505885          0.997142
4  0440221471          0.997106
5  0671568175          0.997091
6  0441104029          0.997069
7  042517736X          0.997041
8  0684854759          0.997036
9  0449006530          0.997036
          ISBN  predicted_rating
1   0786817070          3.800723
2   0060505885          1.317083
3   087113795X          1.212394
4   0316780812          1.207379
5   0441104029          1.205273
6   0449006530          1.204160
7   042517736X          1.198756
9   0684854759          1.171099
10  031286504X          1.169287
11  0671568175          1.164894
         ISBN  predicted_rating
0  067157874X         10.000000
0  0375703063          1.219891
1  0375760911          1.116988
2  0060505885          1.023389
3  0440221471          1.013003
4  0345433491          1.011824
5  0441104

In [285]:
#cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(cf_recommender_model)
print('\nGlobal metrics:\n%s' % global_metrics)
detailed_results_df.head(10)


Global metrics:
{'modelName': 'hybrid', 'recall@5': 0.6124567474048442, 'recall@10': 0.9411764705882353, 'precision@5': 0.05574803149606299, 'precision@10': 0.042834645669291335}


,hits@5_count,hits@10_count,recommended@5_count,recommended@10_count,relevents,recall@5,recall@10,precision@5,precision@10,User-ID
444,5,10,5,10,10,0.500000,1.0,1.0,1.0,11676
628,5,8,5,10,8,0.625000,1.0,1.0,0.8,135149
22,5,8,5,10,8,0.625000,1.0,1.0,0.8,153662
131,4,7,5,10,7,0.571429,1.0,0.8,0.7,16795
316,4,7,5,10,7,0.571429,1.0,0.8,0.7,78834
368,3,5,5,10,5,0.600000,1.0,0.6,0.5,95359
487,2,2,5,10,2,1.000000,1.0,0.4,0.2,167471
391,2,2,5,10,2,1.000000,1.0,0.4,0.2,128208
70,2,2,5,10,2,1.000000,1.0,0.4,0.2,2276
492,2,2,5,10,2,1.000000,1.0,0.4,0.2,30276
